In [2]:
import os
import warnings
warnings.filterwarnings("ignore")
import torch
from torch import Tensor
import torch.nn as nn
import pandas as pd
import numpy as np
import random

In [ ]:
def get_tokens_segments(tokens_a, tokens_b=None):
    # input token_a: list of tokens, [tk1, tk2,...,tkN]
    # input token_b if not none: list of tokens, [tk1, tk2,...,tkN]
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    segments = [0] * len(tokens)
    if tokens_b:
        tokens += tokens_b + ['<sep>']
        segments += [1]*(len(tokens_b) + 1)
    return tokens, segments

In [2]:
x = torch.randn(3, 4, 5)

In [4]:
batch = torch.tensor([0, 0, 1, 1])
pos = torch.tensor([0,1, 0, 3])

In [3]:
x = torch.tensor([0,0,1,1])
x.repeat_interleave(3)

tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])